In [ ]:
!pip install numpy psycopg2 clickhouse_connect

In [ ]:
import psycopg2 as psycopg2
import clickhouse_connect

postgres_conn = psycopg2.connect('host=db.mpkazantsev.ru port=5432 dbname=postgres user=postgres password=5555')
clickhouse_conn = clickhouse_connect.get_client(host='db.mpkazantsev.ru', port=8123, database='maindb',
                                                user='stager', password='stager')


In [ ]:
TYPE_MAPPER = {
    'boolean': 'UInt8',
    'smallint': 'Int16',
    'integer': 'Int32',
    'bigint': 'Int64',
    'real': 'Float32',
    'double precision': 'Float64',
    'numeric': 'Decimal(38, 18)',
    'character varying': 'String',
    'text': 'Text',
    'date': 'Date',
    'timestamp without time zone': 'DateTime',
    'bytea': 'String',
    'json': 'String',
    'jsonb': 'String',
    'uuid': 'String',
    'inet': 'String',
}


def map_pg_to_ch_type(pg_data_type):
    return TYPE_MAPPER.get(pg_data_type, 'String')


def create_table_in_clickhouse(source_table_name, dest_table_name,
                               postgres_conn, clickhouse_conn,
                               order_columns_names):
    pg_cursor = postgres_conn.cursor()
    pg_cursor.execute(f"SELECT column_name, data_type FROM information_schema.columns WHERE table_name = '{source_table_name}'")
    pg_schema = pg_cursor.fetchall()
    create_table_query = f'CREATE TABLE {dest_table_name} ('
    for col_name, pg_data_type in pg_schema:
        ch_data_type = map_pg_to_ch_type(pg_data_type)
        create_table_query += f'{"`" + col_name + "`"} {ch_data_type}, '
    order_columns_in_string = '(' + ','.join(map(lambda x: '`' + x + '`', order_columns_names)) + ')'
    create_table_query = create_table_query.rstrip(', ') + ') ENGINE = MergeTree() ORDER BY' + order_columns_in_string
    print(create_table_query)
    clickhouse_conn.command(create_table_query)

create_table_in_clickhouse('ontime', 'adm_my_ontime2', postgres_conn, clickhouse_conn, ['Year', 'Month', 'DayofMonth'])

In [ ]:
def get_correct_value(value):
    if type(value) in [int, float]:
        return str(value)
    elif value == '':
        return 'Null'
    else:
        return "'" + str(value) + "'"


def insert_increment_from_postgres_into_clickhouse(source_table_name, dest_table_name,
                                         postgres_conn, clickhouse_conn,
                                         start_value, end_value, increament_column_name):
    pg_cursor = postgres_conn.cursor()
    pg_cursor.execute(f"SELECT * FROM {source_table_name} WHERE \"{increament_column_name}\" BETWEEN '{start_value}' AND '{end_value}'")
    pg_data = pg_cursor.fetchall()
    ch_columns = ["`" + desc[0] + "`" for desc in pg_cursor.description]
    ch_values = ','.join(['(' + ','.join(map(get_correct_value, row)) + ')' for row in pg_data[:3]])
    clickhouse_conn.command(f"INSERT INTO {dest_table_name} ({','.join(ch_columns)}) VALUES " + ch_values)


insert_increment_from_postgres_into_clickhouse('public.ontime', 'adm_my_ontime1', postgres_conn, clickhouse_conn, '2017-05-01', '2017-05-02')